# Self-Driving Car Engineer Nanodegree


## Project: **Finding Lane Lines on the Road** 
***
This project provides basic algorithm to detect a lane on a road. 
First the Helper Functions are declared, which consists of the deepest implementation level.
As an abstraction, a pipeline function is implemented that can just be called with one parameter, your image.
The return value is your image with the detected lanes on it.

To see the developed algorithm in action, you can easily execute the apply it on several test images and even videos.

Have fun!
E-Mail: alextreib94@gmail.com

---

**Note: If you encounter a import error or anything like that, make sure you have loaded all previous module. (Just click Run). If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output**

Make sure you have all prerequisites:
https://github.com/udacity/CarND-Term1-Starter-Kit

---

## Helper Functions

Here are some helper functions that help building up the pipeline.

In [17]:
import math
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2

###Parameters###
#Region of interest
left_bottom = [110, 540]
right_bottom = [960, 520]
apex = [480, 310]
vertices=np.array([left_bottom,right_bottom,apex])

#Gaussian blur
kernel=5

#Canny
low_threshold=50 
high_threshold=150

#Hough
rho = 1
theta = np.pi/180
hough_threshold = 1
min_line_len = 3
max_line_gap = 1


def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, np.int32([vertices]), ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    """
    This function draws the lines with an inplace drawing.
    Here, the lines and the mainline are drawn.
    """   
    
    left_edge=[left_bottom,apex]
    right_edge=[right_bottom,apex]
    
    left_lines=[]
    right_lines=[]
    left_points=[]
    right_points=[]
    
    #Separating the line into left and right lines
    for line in lines:
        for x1,y1,x2,y2 in line:
            #Only for formatting
            point_line=[(x1,y1),(x2,y2)]
            element_line=[x1,y1,x2,y2]
            
            #What is closer?
            if(linediff(left_edge,point_line)<linediff(right_edge,point_line)):
            #Closer to left
                left_lines.append([element_line])
                left_points.append([x1,y1])
                left_points.append([x2,y2])
            else:
            #Closer to right
                right_lines.append([element_line])
                right_points.append([x1,y1])
                right_points.append([x2,y2])
    
    #Drawing the mainLanes in red
    draw_mainLane(img,right_points)
    draw_mainLane(img,left_points)
    
    #Drawing the little lines (green for left lane, blue for right lane)
    for line in left_lines:        
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), [0, 255, 0], thickness)
           
    for line in right_lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), [0, 0, 255], thickness)

def draw_mainLane(img,points):
    """
    This function draws the main lane with the given points.
        
    Returns an image with hough lines drawn.
    """
    #Calculate the length of the vector
    x_values=(np.array(points))[:,0]
    y_values=(np.array(points))[:,1]
    x_values.sort()
    y_values.sort()
    
    #Get last 10&
    percent=0.2
    highest_x=x_values[-int(x_values.size*percent):].mean()
    lowest_x=x_values[:int(x_values.size*percent)].mean()
    highest_y=y_values[-int(y_values.size*percent):].mean()
    lowest_y=y_values[:int(y_values.size*percent)].mean()
    
    #Calculate the distance between the two points 
    dist=cv2.norm((highest_x,highest_y), (lowest_x,lowest_y))
    
    [vx,vy,x0,y0] = cv2.fitLine(np.array(points, dtype=np.int32), cv2.DIST_L2,0,0.01,0.01)
    
    #m is the length of the vector
    m=dist*0.5
    x1=int(x0-m*vx)
    y1=int(y0-m*vy)
    x2=int(x0+m*vx)
    y2=int(y0+m*vy)
    
    #Finally, draw the line
    cv2.line(img, (x1, y1), (x2, y2), [255, 0, 0], 30)
            
def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., γ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + γ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, γ)


def linediff(firstline, secondline):
    """
    Calculates the minimum distance between two lines (calculated with perpendicular vector)
    input parameter:
    firstline - line that contains the x1,y1,x2,y2 for one line
    secondline -line that contains the x1,y1,x2,y2 for second line
    
    returns - shortest distance between lines
    """

    #Calculate the cross product to a perpendicular vector 
    (Fx1,Fy1),(Fx2,Fy2) = firstline
    (Sx1,Sy1),(Sx2,Sy2) = secondline
    Fdx,Fdy = Fx2-Fx1,Fy2-Fy1
    Sdx,Sdy = Sx2-Sx1,Sy2-Sy1
    
    perpen_vec_dx,perpen_vec_dy = (Fdy - Sdy, Sdx-Fdx)

    #Perpen_vecp_normalized = perpen_vec / distance of common perp
    perpen_vec_length = math.hypot(perpen_vec_dx,perpen_vec_dy)
    
    perpen_vec_normalized_dx = perpen_vec_dx/float(perpen_vec_length)
    perpen_vec_normalized_dy = perpen_vec_dy/float(perpen_vec_length)

    #step3: length of (pointonline1-pointonline2 dotprod normalized_perp).
    short_vec_dx = (Fx1-Sx1)*perpen_vec_normalized_dx
    short_vec_dy = (Fy1-Sy1)*perpen_vec_normalized_dy

    minimum_distance = math.hypot(short_vec_dx,short_vec_dy)
    
    return minimum_distance

## Lane Finding Pipeline

Build pipeline function that contains that overall dataflow.

In [18]:
def pipeline(origin_img):
    #Main functionality
    gray_img=grayscale(origin_img)
    blurred_img=gaussian_blur(gray_img, kernel)
    masked_edges =canny(blurred_img, low_threshold, high_threshold)
    filtered_masked_edges=region_of_interest(masked_edges, vertices)
    line_img=hough_lines(filtered_masked_edges, rho, theta, hough_threshold, min_line_len, max_line_gap)

    output_img = weighted_img(line_img, origin_img)    
    return output_img    

## Building tests images

Here, the pipeline algorithm will be applied to all the provided test_images

In [19]:
import os
import cv2
import matplotlib.image as mpimg

test_images=os.listdir("test/images/")
output_directory='results/images/'

try:
    os.stat(output_directory)
except:
    os.mkdir(output_directory)       

#Going through every test_image
for file_name in test_images:
    img = cv2.imread('test/images/'+file_name)
    img=pipeline(img)
    cv2.imwrite(output_directory+file_name, img)

## Applying the pipeline on test videos

In the following code snippet, the pipeline will be tested on test/videos.

In [7]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML

def pipeline_handle(image):
    return pipeline(image)

test_videos=os.listdir("test/videos/")

for file_name in test_videos:
    video_fileName = 'test/videos/'+file_name
    fileClip = VideoFileClip(video_fileName)
    clip = fileClip.fl_image(pipeline_handle)

    %time clip.write_videofile('results/videos/'+file_name, audio=False)

[MoviePy] >>>> Building video results/videos/solidWhiteRight.mp4
[MoviePy] Writing video results/videos/solidWhiteRight.mp4


100%|█████████████████████████████████████████████████████████████████████████████▋| 221/222 [00:03<00:00, 66.32it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: results/videos/solidWhiteRight.mp4 

Wall time: 3.73 s
[MoviePy] >>>> Building video results/videos/solidYellowLeft.mp4
[MoviePy] Writing video results/videos/solidYellowLeft.mp4


100%|█████████████████████████████████████████████████████████████████████████████▉| 681/682 [00:10<00:00, 63.10it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: results/videos/solidYellowLeft.mp4 

Wall time: 11.2 s


Now you can your videos in the directory results/videos.
Enjoy watching!

--

The End